In [ ]:
!pip install accelerate -U
!pip install transformers
!pip install datasets
!pip install pandas
!pip install numpy
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments
from transformers import pipeline, Trainer, DataCollatorForTokenClassification
from datasets import load_dataset, load_metric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 123.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.6 MB/s eta 0:00:00


### Datasets
Trong bài này, nhóm em sử dụng bộ dataset "conll2003" có trong thư viện datasets để tiến hành training dữ liệu, validation và test. Nhóm tiến hành giữ nguyên bộ dữ liệu đã được tự động phân chia, và bộ dữ liệu có các thông tin như sau:

In [ ]:
raw_data = load_dataset("conll2003")
raw_data

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

### Model
Tiến hành tạo model từ mẫu đã có sẵn trong thư viện huggingface, một thư viện chuyên dành cho NLP (Xử lý ngôn ngữ tự nhiên). Model này đã được fine-tuned theo chuẩn của bộ dữ liệu "conll2003" ở trên.<br>
Trong lab này, nhóm sử dụng mô hình ngôn ngữ BERT (Bidirectional Encoder Representations from Transformers) được xây dựng bới Google AI.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

### Tokenization

In [ ]:
# Hàm tokenization và gán nhãn tương ứng
def tokenize_and_align_labels(data_set):
    tokenized_inputs = tokenizer(data_set["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(data_set[f"ner_tags"]):
        # Ánh xạ id tới từ tương ứng
        word_ids = tokenized_inputs.word_ids(batch_index = i)
        previous_word_idx = None
        label_ids = []

        # Thiết lập token đặc biệt tới -100
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)

            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_dataset = raw_data.map(
    tokenize_and_align_labels,
    batched = True,
    remove_columns = raw_data["train"].column_names
)

tokenized_dataset


Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

### Training

In [ ]:
!pip install seqeval
metric = load_metric("seqeval")

label_list = raw_data["train"].features[f"ner_tags"].feature.names
labels = [label_list[i] for i in raw_data["train"][0][f"ner_tags"]]

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=7d961b2dde10423daea447b6dfd313286e30ffe4c6f54c2a35426826bede1213
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


<ipython-input-5-593c47f12f26>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [ ]:
label_names = raw_data["train"].features["ner_tags"].feature.names
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

label_names = raw_data["train"].features["ner_tags"].feature.names
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER", id2label = id2label, label2id = label2id)


In [ ]:
training_args = TrainingArguments(
    output_dir = "./Trained_data",
    evaluation_strategy = "epoch",
    save_strategy="epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 3,
    weight_decay = 0.01,
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset["train"],
    eval_dataset = tokenized_dataset["validation"],
    tokenizer = tokenizer,
    data_collator = DataCollatorForTokenClassification(tokenizer = tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()
trainer.save_model('./Saved_model')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.050500,0.050536,0.945244,0.950017,0.947625,0.991083
2,0.004800,0.049564,0.945842,0.955234,0.950515,0.991531
3,0.001900,0.051956,0.950913,0.955234,0.953069,0.991745


In [ ]:
model_checkpoint = "./Saved_model"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
temp = token_classifier("Hello, my name is Joe and I live in Los Angeles")
print(temp)

[{'entity_group': 'PER', 'score': 0.99906904, 'word': 'Joe', 'start': 18, 'end': 21}, {'entity_group': 'LOC', 'score': 0.999501, 'word': 'Los Angeles', 'start': 36, 'end': 47}]


### Output JSON file

In [ ]:
import json

json_object = json.dumps(str(temp), indent = 4)
with open("output.json", 'w') as file:
    file.write(json_object)
file.close()